In [ ]:
import gym
env    = gym.make('LunarLander-v2')
state  = env.reset()
action = env.action_space
env = env.unwrapped

# Policy gradient has high variance, seed for reproducability
env.seed(1)

print("env.action_space", env.action_space)
print("env.observation_space", env.observation_space)
print("env.observation_space.high", env.observation_space.high)
print("env.observation_space.low", env.observation_space.low)
action_step = [0.00941777229309082, -0.0044043660163879395, 0.057526230812072754, 0.009538531303405762]
action_step = np.random.choice(len(range(4)))
state, reward, done, _ = env.step(action_step)
print(action)
print(state)
print(reward)
env.render()
#env.close()

In [ ]:
env.close()

In [ ]:
from Agents.random_agent import RandomAgent

In [ ]:
action_space = 

runtime = 5.                                     # time limit of the episode
init_pose = np.array([0., 0., 10., 0., 0., 0.])  # initial pose

In [ ]:
RandomAgent(action)

In [ ]:
import random

class Basic_Agent():
    def __init__(self, task):
        self.task = task
    
    def act(self):
        new_thrust = random.gauss(450., 25.)
        return [new_thrust + random.gauss(0., 1.) for x in range(4)]

1. Import the Necessary Packages

In [ ]:
import gym
import random
import keras as K
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

2. Instantiate the Environment and Agent

In [1]:
#Load Main Modules
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras import backend as K
from collections import deque
from itertools import count
import time
#from sklearn.preprocessing import normalize

#Load Graphical Libraries
import matplotlib.pyplot as plt
%matplotlib notebook

# Load Custom Modules
from Task.SoftLanding import SoftLanding
from Agents.agent import DDPG

# Check GPU compatibility 
#K.tensorflow_backend._get_available_gpus()
#config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 70} ) 
#sess = tf.Session(config=config) 
#K.tensorflow_backend.set_session(sess)

########################################################

num_episodes =700 # number of episodes

#Create agent
Q_targets_next = ([])
Q_targets      = ([])

random_seed = 0

def train(num_episodes=300, max_t=20000, print_every=5):
    
    # Initialize training
    task = SoftLanding()
    agent = DDPG(task, random_seed)

    #####################
    RENDER_ENV = False
    score_average  = []
    score_average2 = []
    scores = []
    best_score = -np.inf
    best_episode_states = []
    best_episode_actions = []

    
    for episode in range(num_episodes + 1):

        # Initialize episode
        state = agent.reset_episode()
        done = False
        state = state.reshape(1,-1)  
        score = 0.0
        steps = 0
        tic = time.time()
        
        episode_states = []
        episode_actions = []
        
        # Execute
        for t in range(max_t):
            if (RENDER_ENV): task.env.render()
            action = agent.act(state)
            #print(action)
             
            # Turn off engines if lander touch the ground   
            if ((state[-1,6] != 1.0) and (state[-1,7] != 1.0)) :
                # Evaluate possible actions
                action = agent.act(state)
                #print(action)
                action = np.clip(action, task.action_low, task.action_high)
            else:
                #Turn off the engine whe
                action = np.zeros(agent.action_size)
                
            # Apply actions to the environment
            next_state, reward, done = task.step(action)
            next_state = next_state.reshape(1,-1) 
            # Learn action/state pair
            agent.step(action, reward, next_state, done, t)
            # Move to the next state
            state = next_state
            # Calculate Total reward of the episode
            score += reward
            steps += 1
            
            #Store information for debugging and plotting
            episode_actions.append(action)
            episode_states.append(state)
         
            Q_targets_next.extend([agent.Q_targets_next])
            Q_targets.append(agent.Q_targets)           
            
            # Stopping conditions
            if done:

                break 
                
            if score < -300:
                break
                
            # Lander angle is outside the control limits
            if (state[-1,4] < -0.7 or state[-1,4] > 0.7):
                break    

            toc = time.time()
            elapsed_sec = toc - tic
            
            #Timeout condition
            if elapsed_sec > 60:
                done = True  
                
        #Store cumulative reward for this episode        
        scores.append(score)   
        avg  = sum(scores)/max(1,len(scores))
        avg2 = score/max(1,steps)
        score_average.append(avg)
        score_average2.append(avg2)

        
        if score > best_score:
            best_score = score
            episode_states = episode_states
            episode_actions = episode_actions
            
        if score >= 200.0:            
            print('\nEnvironment solved in {:d} episodes!\t Score: {:.2f}'.format(episode, score)) 
            print('\rEpisode {}, Score: {:.2f}, Best: {:.2f}, Min: {:.2f}, Time: {:.2f}'\
                      .format(episode, score, best_score, np.min(scores), elapsed_sec), end="\n")
            print(action)
            print(next_state)
            #print(episode_states)
            #print(episode_actions)
            #print(reward)
            print(agent.epsilon)
            agent.epsilon  = agent.epsilon * 0.8
            #break    
            
            
        if episode % print_every == 0:
            print('\rEpisode {}, Score: {:.2f}, Best: {:.2f}, Min: {:.2f}, Time: {:.2f}'\
                      .format(episode, score, best_score, np.min(scores), elapsed_sec), end="\n")
            print(action)
            print(next_state)
            #print(reward)
            print(agent.epsilon)
            
        
    
    return scores, score_average, score_average2


scores, score_average, score_average2 = train(num_episodes)



Using TensorFlow backend.


-----------------------------------
Environment Observation_space:  Box(8,)
Environment Action_space:  Box(2,)
-----------------------------------

Episode 0, Score: -103.31, Best: -103.31, Min: -103.31, Time: 0.01
[-0.44402547 -0.25391882]
[[ 0.3974968   1.0882643   1.0180988  -0.7396019  -0.7363728  -0.47008592
   0.          0.          0.40750265  1.0712651   1.0180526  -0.7663034
  -0.75987625 -0.47006798  0.          0.          0.41751185  1.0536712
   1.018005   -0.7930043  -0.7833788  -0.4700501   0.          0.        ]]
1.0
Episode 5, Score: -302.33, Best: -103.31, Min: -392.37, Time: 1.10
[ 0.41751023 -0.21738146]
[[-0.5518091   2.7834702  -1.2158413   0.99474335  0.4175444   0.10825054
   0.          0.         -0.5639896   2.8057427  -1.2232997   0.98832124
   0.42322347  0.11358196  0.          0.         -0.57633924  2.827977
  -1.2402025   0.9865984   0.42890012  0.11353301  0.          0.        ]]
0.9997100000000013
Episode 10, Score: -25.60, Best: -25.60, Min: -392.

Episode 100, Score: -102.25, Best: -25.60, Min: -392.37, Time: 0.29
[-0.07739009  1.        ]
[[ 0.04975996  1.5842799   0.35253543 -0.06115698 -0.79000527 -1.1043776
   0.          0.          0.05295486  1.582815    0.35953364 -0.09348753
  -0.8477062  -1.1540215   0.          0.          0.05623026  1.5806867
   0.36665487 -0.12580474 -0.90792364 -1.2043508   0.          0.        ]]
0.9890500000000498
Episode 105, Score: -104.39, Best: -25.60, Min: -392.37, Time: 0.33
[0.20521534 0.87548611]
[[ 0.004708    1.355753    0.2702133  -0.30969268 -0.7160861  -1.0321658
   0.          0.          0.00753288  1.3492862   0.32251096 -0.3117849
  -0.7695948  -1.0701761   0.          0.          0.01089869  1.3426865
   0.37593275 -0.3200196  -0.8250298  -1.1087028   0.          0.        ]]
0.9885200000000522
Episode 110, Score: -75.19, Best: -25.60, Min: -392.37, Time: 0.34
[-0.0981945   0.75468539]
[[-0.07816086  1.5260237  -0.03751316 -0.09203637 -0.8161506  -1.082521
   0.          0.   

Episode 205, Score: -119.63, Best: -20.86, Min: -392.37, Time: 0.30
[ 0.15894125 -0.5401339 ]
[[-0.12080546  1.5397329  -0.6661553  -0.04710298  0.72719985  1.0237274
   0.          0.         -0.12748976  1.5391908  -0.7072243  -0.04822817
   0.7795702   1.0474102   0.          0.         -0.1345644   1.5385084
  -0.74511385 -0.05635353  0.8331091   1.0707786   0.          0.        ]]
0.9782900000000988
Episode 210, Score: -138.39, Best: -20.86, Min: -392.37, Time: 0.33
[-0.1502714 -1.       ]
[[-0.08893614  1.4239364  -0.5469104  -0.16000274  0.75857544  1.053746
   0.          0.         -0.09409237  1.4202176  -0.5551978  -0.19155413
   0.8137145   1.1027827   0.          0.         -0.09932442  1.4158424
  -0.5620864  -0.22333866  0.8711817   1.1493468   0.          0.        ]]
0.977800000000101
Episode 215, Score: -136.69, Best: -20.86, Min: -392.37, Time: 0.36
[ 0.06971745 -1.        ]
[[-0.10496245  1.607915   -0.6360353   0.08821569  0.7688947   1.0805999
   0.          0.  

Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting 

...

Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting 

Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting 

Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting 

Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting Memory
Overwriting 

KeyboardInterrupt: 

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(score_average)+1), score_average)
plt.ylabel('Average Score')
plt.xlabel('Episode #')
plt.show()

In [ ]:
smooth = 5
plt.figure(figsize=(10,5))
plt.plot(scores, '.', alpha=0.25, color='xkcd:blue')
plt.plot(np.convolve(scores, np.ones(smooth)/smooth)[(smooth-1)//2:-smooth], 
         color='xkcd:blue', 
         label='Total Reward')
plt.ylabel('Total Reward')
plt.legend(loc=2)



plt.xlabel("Episode")
plt.xlim(0, len(scores))
plt.ylim(np.mean(scores)-5*np.std(scores), np.mean(scores)+5*np.std(scores))
plt.show()



In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(score_average2)+1), score_average2)
plt.ylabel('Average Score')
plt.xlabel('Episode #')
plt.show()

In [ ]:
import matplotlib.mlab as mlab
from scipy.stats import norm
#%matplotlib inline

mu    = np.mean(scores)
sigma = np.std(scores)

num_bins = 40

fig2 = plt.figure()
plt.style.use('seaborn-white')
n, bins, patches = plt.hist(scores, num_bins,  alpha=0.9, color='steelblue', edgecolor='black')

# add a 'best fit' line
#y = norm.pdf(bins, mu, sigma)
#plt.plot(bins, y, 'r--')
plt.xlabel('Rewards')
plt.ylabel('Frequency')
plt.title(r'Rewards Histogram: $\mu={:.2f}$, $\sigma={:.2f}$'.format( mu, sigma))
 
# Tweak spacing to prevent clipping of ylabel
#plt.subplots_adjust(left=0.15)
plt.show()

In [ ]:
Q_targets_next_flat = []
for sublist in Q_targets_next[128:]:
    for val in sublist:
        Q_targets_next_flat.append(val)
Q_targets_flat = []
for sublist in Q_targets[128:]:
    for val in sublist:
        Q_targets_flat.append(val)

In [ ]:
fig = plt.figure()
bx = fig.add_subplot(111)

plt.scatter(Q_targets_flat, Q_targets_next_flat)
plt.ylabel('Q Target Prediction')
plt.xlabel('Q Target Calculated')
plt.show()

In [ ]:
fig = plt.figure()
bx = fig.add_subplot(111)
plt.plot(Q_targets_next_flat, 'r', alpha=0.4) 
plt.plot(Q_targets_flat,'g',alpha=0.3)

plt.ylabel('Q Value')
plt.xlabel('Iteration')
plt.show()

In [ ]:
sys.stdout.flush()
